In [26]:
import cantera as ct
import numpy as np
import pandas as pd

In [27]:
#Assumptions
Pressure_Ratio = 26
comp_efficiency = 0.85
Turbine_Inlet_Temperature = 1700 #max inlet temperature in kelvin
turb_efficiency = 0.92

P6 = 101325 #outlet pressure in pa

In [28]:
#Analyze compressor

#Air inlet to compressor
state1 = ct.Solution('gri30.xml') #usegri30.xml gas definitions

#First edit the composition of the gas
phi = 2.756
state1.X = {'O2':3*phi, 'N2':3*3.76*phi}

#Next edit the states thermodynamic state
state1.TP = 298, 101325

In [30]:
#isentropically compressed air
state2s = ct.Solution('gri30.xml') #usegri30.xml gas definitions

#Composition does not change
state2s.X = state1.X
#Set entropy and pressure for isentropic compression
state2s.SP = state1.s, state1.P*Pressure_Ratio

state2s

In [33]:
#Use efficiency to calculate state 2

#Air outlet to compressor
state2 = ct.Solution('gri30.xml') #usegri30.xml gas definitions

#Composition does not change
state2.X = state1.X
#Calculate enthalpy
h2 = ((state2s.h-state1.h)/comp_efficiency)+state1.h

#Set enthalpy and pressure for state 2
state2.HP = h2,state1.P*Pressure_Ratio

state2()


  gri30:

       temperature         809.762  K
          pressure     2.63445e+06  Pa
           density          11.289  kg/m^3
  mean mol. weight         28.8507  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      5.3811e+05        1.552e+07     J
   internal energy      3.0475e+05        8.792e+06     J
           entropy          6989.5        2.017e+05     J/K
    Gibbs function     -5.1217e+06       -1.478e+08     J
 heat capacity c_p          1107.8        3.196e+04     J/K
 heat capacity c_v          819.58        2.365e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2       0.210084         0.233008         -24.3256
                N2       0.789916         0.766992         -21.3147
     [  +51 minor]              0                0



In [16]:
#Define State 3
#Fuel stream into compressor
state3 = ct.Solution('gri30.xml') #usegri30.xml gas definitions

#One mole of fuel C2H5
state3.X = {'C2H4':1}
#Set pressure and temperature. Pressure will be the same as state 2, temperature at 298K
state3.TP = 298, state2.P



In [17]:
#Mix states 2 and 3 using variable Cp and find temperature before combustion. Iterate to find temperature
T_R = 786.4


In [18]:
#Define staterR as the reactants
stateR = ct.Solution('gri30.xml') #usegri30.xml gas definitions
phi = 2.756
stateR.X = {'C2H4':1, 'O2':3*phi, 'N2':3.76*3*phi}
#Using temperature solve above
stateR.TP = T_R, state2.P

In [19]:
#Analyze Turbine

#State 4 as turbine inlet
#State with state R, equililbrate through temperature and pressure
state4 = ct.Solution('gri30.xml') #usegri30.xml gas definitions

state4.X = stateR.X
state4.TP = 1700, state2.P

state4.equilibrate('TP')

state4.s

8009.661348119622

In [21]:
#Isentropic efficiency

#Turbine outlet
state5s = ct.Solution('gri30.xml') #usegri30.xml gas definitions

#Start with the reactants and eqilibrate to match temperature with entropy value at outlet pressure
state5s.X = stateR.X

#Iterate temperature to match entropy
T= state4.T

#Create bool n
n = 'False'

while n is 'False':
    #Set state for elquilibrium
    state5s.TP = T, 101325  
    #Equilibrate
    state5s.equilibrate('TP')
    #Calculate and print delta_s
    delta_s = state4.s - state5s.s
    
    if delta_s > 5:
        T = T + 1
    elif delta_s < -5:
        T = T - 1
    else:
        n = 'True'
           
#flame equilibruim temp
state5s.T

795.9999999999993

In [22]:
#Use turbine efficiency to calculate actual temperature

#Turbine outlet
state5 = ct.Solution('gri30.xml') #usegri30.xml gas definitions
state5.X = stateR.X

h5 = state4.h - turb_efficiency*(state4.h - state5s.h)

h5

#Set state for elquilibrium
state5.TP = 871.6, 101325  
#Equilibrate
state5.equilibrate('TP')

state5.h


-465373.3332727448

In [23]:
h5

-462762.7637137661

In [24]:
(state4.h - state5s.h)

1125173.6738690122

In [ ]:
#iterate to match h5
#Create bool n
n = 'False'

T = state5s.T

while n is 'False':
    #Set state for elquilibrium
    state5.TP = 500, 101325  
    #Equilibrate
    state5.equilibrate('TP')
    #Calculate and print delta_s
    delta_h = h5 - state5.h
    
    if delta_h > 100:
        T = T - 1
    elif delta_h < -100:
        T = T + 1
    else:
        n = 'True'